# **1.Import library**

In [25]:
import pandas as pd
import numpy as np
import re

Menghubungkan Google Colab dengan Google Drive
# **2.Menghubungkan Google Colab dengan Google Drive**

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **3.Membaca data set kotor**

In [27]:
df = pd.read_csv('/content/drive/MyDrive/DATA_MINING_2318082/Data_set_kotor/Phone_ratings.csv')

# **4.Menampilkan 10 baris pertama**

In [28]:
print(df.head(10))

                                         dirty title  ratings averageRating  \
0  Redmi A1 (Light Blue, 2GB RAM, 32GB Storage) |...    7,009           4.0   
1  Redmi 9 Activ (Carbon Black, 4GB RAM, 64GB Sto...  311,978           4.2   
2  Redmi 9A Sport (Coral Green, 2GB RAM, 32GB Sto...  311,978           4.2   
3  OPPO A31 (Mystery Black, 6GB RAM, 128GB Storag...   58,291           4.2   
4  Redmi 10A (Slate Grey, 4GB RAM, 64GB Storage) ...   12,043           3.9   
5  OPPO A74 5G (Fantastic Purple,6GB RAM,128GB St...   32,469           4.2   
6  realme narzo 50i (Carbon Black, 2GB RAM+32GB S...   30,328           4.0   
7  Samsung Galaxy M13 (Aqua Green, 4GB, 64GB Stor...   17,760           4.0   
8  OnePlus Nord CE 2 Lite 5G (Blue Tide, 6GB RAM,...  105,027           4.3   
9  Redmi 10A Sport (Slate Grey, 6GB RAM, 128GB St...      812           3.7   

    price  
0   6,199  
1          
2          
3          
4          
5          
6   6,999  
7  11,999  
8          
9         

# **5.Mengecek kondisi awal dataset**

In [30]:
print("Jumlah Data:", df.shape[0])
print("Jumlah Kolom:", df.shape[1])
print("\nInfo Dataset:")
print(df.info())

print("\nTipe Data per Kolom:")
print(df.dtypes)


Jumlah Data: 1536
Jumlah Kolom: 4

Info Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1536 entries, 0 to 1535
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   dirty title    1536 non-null   object
 1   ratings        1536 non-null   object
 2   averageRating  1536 non-null   object
 3   price          1536 non-null   object
dtypes: object(4)
memory usage: 48.1+ KB
None

Tipe Data per Kolom:
dirty title      object
ratings          object
averageRating    object
price            object
dtype: object


# **6.Data cleansing menstandarisasi column dirty title**

In [40]:
df['brand'] = df['dirty title'].str.split().str[0]

df['brand'] = df['brand'].str.strip().str.title()

# Menampilkan 10 brand unik pertama
print(df['brand'].unique()[:10])


['Redmi' 'Oppo' 'Realme' 'Samsung' 'Oneplus' 'Tecno' 'Motorola' 'Nokia'
 'Iqoo' 'Shivansh']


# **7.Data cleansing menstandarisasi column type**

In [32]:
df['type'] = df['dirty title'].str.split().str[1]
df['type'] = df['type'].str.strip().str.title()
print(df['type'].unique()[:10])


['A1' '9' '9A' 'A31' '10A' 'A74' 'Narzo' 'Galaxy' 'Nord' 'Spark']


# **8.Data cleansing menstandarisasi column rating**

In [33]:
def clean_rating(rating):
    if pd.isna(rating):
        return np.nan
    rating = str(rating).strip()
    rating = rating.lower()

    match = re.search(r'\d+(\.\d+)?', rating)
    if match:
        return float(match.group())

    word_to_num = {
        'zero': 0, 'one': 1, 'two': 2, 'three': 3, 'four': 4,
        'five': 5, 'six': 6, 'seven': 7, 'eight': 8, 'nine': 9, 'ten': 10
    }
    for word, num in word_to_num.items():
        if word in rating:
            return float(num)

    return np.nan
df['ratings_clean'] = df['ratings'].apply(clean_rating)
print("10 data ratings asli vs ratings_clean:")
print(df[['ratings', 'ratings_clean']].head(10))


10 data ratings asli vs ratings_clean:
   ratings  ratings_clean
0    7,009            7.0
1  311,978          311.0
2  311,978          311.0
3   58,291           58.0
4   12,043           12.0
5   32,469           32.0
6   30,328           30.0
7   17,760           17.0
8  105,027          105.0
9      812          812.0


# **9.Data cleansing menstandarisasi column average**

In [34]:
df['averageRating'] = pd.to_numeric(df['averageRating'], errors='coerce')
print("Average Rating unik:")
print(df['averageRating'].unique()[:10])

Average Rating unik:
[4.  4.2 3.9 4.3 3.7 3.8 3.1 3.2 4.1 3.4]


# **10.Data cleansing menstandarisasi column price**

In [35]:
df['price'] = df['price'].astype(str).str.replace('[^0-9]', '', regex=True)
df['price'] = pd.to_numeric(df['price'], errors='coerce')

print("\nPrice unik:")
print(df['price'].unique()[:10])



Price unik:
[ 6199.    nan  6999. 11999. 13999.  7559. 17999.  8999. 22999. 10499.]


# **11.Data cleansing mendeteksi data duplikat**

In [36]:
duplicate_rows = df[df.duplicated(
    ['dirty title', 'brand', 'type', 'averageRating', 'price'], keep='first'
)]

print("Jumlah data yang duplikat:", len(duplicate_rows))
duplicate_rows


Jumlah data yang duplikat: 2


,dirty title,ratings,averageRating,price,brand,type,ratings_clean
1094,Casotec Back Cover for Mobile (Plastic_Multico...,56,4.1,149.0,Casotec,Back,56.0
1312,Casotec Back Cover for Mobile (Plastic_Multico...,186,3.9,149.0,Casotec,Back,186.0


# **12.Menghapus data duplikat**

In [37]:
df = df.drop_duplicates(
    subset=['dirty title', 'brand', 'type', 'averageRating', 'price'], keep='first'
)
print("Jumlah data setelah deduplikasi:", len(df))


Jumlah data setelah deduplikasi: 1534


# **13.Data Enrichment**

In [43]:

# --- Data Cleansing - Data Setelah Deduplikasi ---
print(df.head(10))

# --- Data Enrichment berdasarkan averageRating ---
def rating_category(rating):
    if pd.isna(rating):
        return "Unknown"
    elif rating >= 4.5:
        return "Excellent"
    elif rating >= 4.0:
        return "Good"
    elif rating >= 3.0:
        return "Average"
    else:
        return "Poor"

df['rating_category'] = df['averageRating'].apply(rating_category)

# Tampilkan hasil enrichment
df.head(10)



                                         dirty title  ratings  averageRating  \
0  Redmi A1 (Light Blue, 2GB RAM, 32GB Storage) |...    7,009            4.0   
1  Redmi 9 Activ (Carbon Black, 4GB RAM, 64GB Sto...  311,978            4.2   
2  Redmi 9A Sport (Coral Green, 2GB RAM, 32GB Sto...  311,978            4.2   
3  OPPO A31 (Mystery Black, 6GB RAM, 128GB Storag...   58,291            4.2   
4  Redmi 10A (Slate Grey, 4GB RAM, 64GB Storage) ...   12,043            3.9   
5  OPPO A74 5G (Fantastic Purple,6GB RAM,128GB St...   32,469            4.2   
6  realme narzo 50i (Carbon Black, 2GB RAM+32GB S...   30,328            4.0   
7  Samsung Galaxy M13 (Aqua Green, 4GB, 64GB Stor...   17,760            4.0   
8  OnePlus Nord CE 2 Lite 5G (Blue Tide, 6GB RAM,...  105,027            4.3   
9  Redmi 10A Sport (Slate Grey, 6GB RAM, 128GB St...      812            3.7   

      price    brand    type  ratings_clean rating_category  
0   61990.0    Redmi      A1            7.0            Go

,dirty title,ratings,averageRating,price,brand,type,ratings_clean,rating_category
0,"Redmi A1 (Light Blue, 2GB RAM, 32GB Storage) |...","7,009",4.0,61990.0,Redmi,A1,7.0,Good
1,"Redmi 9 Activ (Carbon Black, 4GB RAM, 64GB Sto...","311,978",4.2,NaN,Redmi,9,311.0,Good
2,"Redmi 9A Sport (Coral Green, 2GB RAM, 32GB Sto...","311,978",4.2,NaN,Redmi,9A,311.0,Good
3,"OPPO A31 (Mystery Black, 6GB RAM, 128GB Storag...","58,291",4.2,NaN,Oppo,A31,58.0,Good
4,"Redmi 10A (Slate Grey, 4GB RAM, 64GB Storage) ...","12,043",3.9,NaN,Redmi,10A,12.0,Average
5,"OPPO A74 5G (Fantastic Purple,6GB RAM,128GB St...","32,469",4.2,NaN,Oppo,A74,32.0,Good
6,"realme narzo 50i (Carbon Black, 2GB RAM+32GB S...","30,328",4.0,69990.0,Realme,Narzo,30.0,Good
7,"Samsung Galaxy M13 (Aqua Green, 4GB, 64GB Stor...","17,760",4.0,119990.0,Samsung,Galaxy,17.0,Good
8,"OnePlus Nord CE 2 Lite 5G (Blue Tide, 6GB RAM,...","105,027",4.3,NaN,Oneplus,Nord,105.0,Good
9,"Redmi 10A Sport (Slate Grey, 6GB RAM, 128GB St...",812,3.7,NaN,Redmi,10A,812.0,Average


# **14.Menyimpan data**


In [39]:
folder_path = '/content/drive/MyDrive/DATA_MINING_2318082/Data_set_bersih/'

df_cleaned.to_csv(folder_path + 'cleaned_data_phone_ratings.csv', index=False)
print(f"File berhasil disimpan di {folder_path}")


File berhasil disimpan di /content/drive/MyDrive/DATA_MINING_2318082/Data_set_bersih/
